# Polyglot Demo - Test

This is a .Net Interactive notebook, initially run in VSCode, but works in Jupyter too.<br> 
The main Kernel (shown upper right) is `.NET Interactive`<br>
Magic commands may be used to switch between languages.<hr>

We can add the Python kernel to .NET interactive - but only if it is not already there!
Similarly, Julia and R could be added.

In [ ]:
// #!connect jupyter --kernel-name pythonkernel --kernel-spec python3



The next cell runs a magic command that lists the magic commands.

[Reference](https://code.visualstudio.com/docs/languages/polyglot)

In [ ]:
#!lsmagic

## Mermaid diagrams work too

In [ ]:
#!mermaid

graph TD;
A --> B;

## F# code

In [ ]:
#!fsharp
let me = "F#"
// that was a magic string to select the cell language
printfn $"Hello from {me}!"

## HTML


In [ ]:
#!html
<div>
  <hr>
   <h4>Hello World from Html</h4>
  <hr>
</div>


##  Http Request

Just define the request and you will get back the results

In [ ]:
#!http

https://icanhazdadjoke.com/



## Javascript

This example also demonstrates how to share variables between kernels.

In [ ]:
#!js
// the hello world program
console.log('Hello World from JavaScript');
// and we can access other kernels' variables
#!set --value @fsharp:me --name meJs
console.log('and from ' + meJs + '!')

## KQL (Kusto Query Language)

[Reference](https://learn.microsoft.com/en-us/azure/data-explorer/kusto/query/)

In [ ]:
#!kql

## Powershell

Also capable of reading from other kernels

In [ ]:
#!pwsh
#!set --value @fsharp:me --name mePwsh
dir
echo "Hello from PowerShell"
echo "and from $mePwsh! "

# SQL

[Reference](https://devblogs.microsoft.com/dotnet/net-interactive-with-sql-net-notebooks-in-visual-studio-code/)

[Ref for Magic commands](https://github.com/dotnet/interactive/blob/main/docs/magic-commands.md)

The sql piece is not behaving well - maybe okay on a local db but has issues with spaces in the connection string.

Recommend using either C# with a sql connection, or sql inside of python (both described below).

## SQL data via Python

Refer to the Visier Part B sample. Great with local dbs, plus you can use visualization tools.

## SQL data via CSharp

This is my preference (other than using a dedicated library).

Have to load the libraries first of course.

In [1]:
#r "nuget: Microsoft.Data.SqlClient, 5.1.4"
#r "nuget: Newtonsoft.Json, 13.0.3"



Installed Packages Microsoft.Data.SqlClient, 5.1.4 Newtonsoft.Json, 13.0.3

In [2]:
using Microsoft.Data.SqlClient;
using Newtonsoft.Json;
using System.IO;

In [3]:
// verify the connection credentials work
internal string credentials  = System.Environment.GetEnvironmentVariable("NIX_DB_DG");

try
{
    using (var connection = new Microsoft.Data.SqlClient.SqlConnection(credentials))
    {   connection.Open();
        var x = connection.State;
        Console.WriteLine($"Connection: {x} succeeded."); 
    }
}
catch (Exception ex)
{
    Console.WriteLine($"{ex.Message}"); 
}

Connection: Open succeeded.


In [4]:
// A function to prettify json
internal string Prettify(string json)
{
   return Newtonsoft.Json.Linq.JToken.Parse(json).ToString(Newtonsoft.Json.Formatting.Indented);
}
// Let's define some classes to manage our schema information

public class TableDefinition
{
    public string SchemaName { get; set; }
    public string TableName {get; set;}
    public List<ColumnDefinition> Columns {get; set; }
}
public class ColumnDefinition
{
    public string ColumnName {get; set; }
    public string DataType { get; set; }
    public bool IsNullable {get; set; }
    public int Size { get; set; }
}
// A function to retrieve schema information for one table
internal string GetTableSchema(string schemaName, string tableName)
{
    string sql = 
        $"select '{schemaName}' SchemaName, '{tableName}' TableName, "
        +"(select column_name ColumnName, data_type DataType, "
        +"case when is_nullable='yes' then 1 else 0 end IsNullable, "
        +"isnull(character_maximum_length, 0) Size "
        +$"from information_schema.columns where TABLE_SCHEMA = '{schemaName}' and "
        +$"TABLE_NAME = '{tableName}' order by ordinal_position "
        +"for json path) Columns for json path, "
        +"WITHOUT_ARRAY_WRAPPER"; // because we are only getting a single result

    // Console.WriteLine(sql);
    using (var connection = new SqlConnection(credentials))
    {
        using (var command = new SqlCommand(sql, connection))
        {
            connection.Open();
            var result = new StringBuilder();
            var reader = command.ExecuteReader();
            if (!reader.HasRows)
            {
                result.Append("{}");// empty object
            }
            else
            {
                while (reader.Read())
                {
                    result.Append(reader.GetValue(0).ToString());
                }
            }
            return result.ToString();
        }
    }
}
    
public TableDefinition GetTableDefinition(string schemaName, string tableName)
{
   // Test against a single table
   var x = GetTableSchema(schemaName, tableName);
   // Console.WriteLine(Prettify(x));
   return Newtonsoft.Json.JsonConvert.DeserializeObject<TableDefinition>(x);
}

In [5]:
string[] tablenames = new string[]
{
    "Content",
    "ContentDurationInMinutes",
    "DailyUserActivity",
    "DefaultDateRangeOptions",
    "LearningCampaignEventCounts",
    "MessageQueue",
    "MessageQueue_PreProcessing",
    "PathwayInsightsKPIs",
    "PopularContentTags",
    "PopularContentTags_Switch",
    "PopularContentTags_Switch",
    "PopularContentTags_Switch_Empty",
    "PopularSearchTerms",
    "PopularUserTags",
    "RelatedContent",
    "RelatedContent_Partition",
    "ReportActivityLog",
    "ReportAggregateTypes",
    "ReportCategories",
    "ReportColumns",
    "ReportDataTypeOperators",
    "ReportDataTypes",
    "ReportDefinitionColumns",
    "ReportDefinitionCustomAttributes",
    "ReportDefinitions",
    "ReportDownloads",
    "ReportExecutionLog",
    "ReportFailureTypeRegexMatches",
    "ReportFailureTypes",
    "ReportFields",
    "ReportFileLog",
    "ReportInstances",
    "ReportLogMetadata",
    "ReportLogs",
    "ReportMethods",
    "ReportOperatorTypes",
    "Reports",
    "ReportSchedule",
    "ReportSearchableTermTypes",
    "ReportTestBatch",
    "ReportTestBatchLog",
    "ReportTimeRanges",
    "ReportTransmissionLogs",
    "ReportTransmitterConfigLinks",
    "ReportTransmitterConfigs",
    "ReportTypes",
    "ReportUserSegments",
    "Salty",
    "SubReports",
    "TableSyncTracking",
    "TransmissionConfig",
    "TrendingContent",
    "TrendingContent_Partition",
    "UserCollectionsStatus",
    "UserContentView_Archive",
    "UserDailyContentViewCount",
    "UserInputCompletions",
    "UserLogins",
    "UserSearchTerms",
    "UserSummary",
    "UserViewBrowsers",
    "UserViewLocations",
    "UserViewOSs",
    "UserViewPlatforms"
};
var tableDefinitions = new List<TableDefinition>();
foreach (string tableName in tablenames)
{ 
    tableDefinitions.Add(GetTableDefinition("rpt", tableName));
}


In [6]:
tableDefinitions.Display();

index value 0 Submission#4+TableDefinition SchemaName rpt TableName Content Columns index value 0 Submission#4+ColumnDefinition ColumnName ContentId DataType bigint IsNullable False Size 0 1 Submission#4+ColumnDefinition ColumnName InputTypeId DataType tinyint IsNullable False Size 0 2 Submission#4+ColumnDefinition ColumnName Title DataType nvarchar IsNullable True Size 2000 3 Submission#4+ColumnDefinition ColumnName Url DataType nvarchar IsNullable True Size 2000 4 Submission#4+ColumnDefinition ColumnName UrlHash DataType binary IsNullable True Size 32 5 Submission#4+ColumnDefinition ColumnName ImageUrl DataType nvarchar IsNullable True Size 2000 6 Submission#4+ColumnDefinition ColumnName DateCreated DataType datetime2 IsNullable True Size 0 7 Submission#4+ColumnDefinition ColumnName DateModified DataType datetime2 IsNullable True Size 0 8 Submission#4+ColumnDefinition ColumnName ExternalId DataType nvarchar IsNullable True Size 2000 9 Submission#4+ColumnDefinition ColumnName ProviderId DataType bigint IsNullable True Size 0 10 Submission#4+ColumnDefinition ColumnName DurationUnit DataType decimal IsNullable True Size 0 11 Submission#4+ColumnDefinition ColumnName DurationUnitType DataType nvarchar IsNullable True Size 100 12 Submission#4+ColumnDefinition ColumnName Obsolete DataType tinyint IsNullable True Size 0 13 Submission#4+ColumnDefinition ColumnName OrganizationId DataType bigint IsNullable True Size 0 14 Submission#4+ColumnDefinition ColumnName FileManaged DataType tinyint IsNullable True Size 0 15 Submission#4+ColumnDefinition ColumnName SYS_CHANGE_VERSION DataType bigint IsNullable False Size 0 16 Submission#4+ColumnDefinition ColumnName SYS_CHANGE_OPERATION DataType nchar IsNullable False Size 1 17 Submission#4+ColumnDefinition ColumnName InputSubTypeId DataType tinyint IsNullable True Size 0 18 Submission#4+ColumnDefinition ColumnName InputKey DataType bigint IsNullable True Size 0 19 Submission#4+ColumnDefinition ColumnName ProviderSourceId DataType bigint IsNullable True Size 0 (2 more) 1 Submission#4+TableDefinition SchemaName rpt TableName ContentDurationInMinutes Columns index value 0 Submission#4+ColumnDefinition ColumnName DurationIso DataType nvarchar IsNullable False Size 20 1 Submission#4+ColumnDefinition ColumnName DurationInMinutes DataType bigint IsNullable True Size 0 2 Submission#4+TableDefinition SchemaName rpt TableName DailyUserActivity Columns index value 0 Submission#4+ColumnDefinition ColumnName ActivityDate DataType date IsNullable False Size 0 1 Submission#4+ColumnDefinition ColumnName FormalCourses DataType int IsNullable True Size 0 2 Submission#4+ColumnDefinition ColumnName InformalCourses DataType int IsNullable True Size 0 3 Submission#4+ColumnDefinition ColumnName Books DataType int IsNullable True Size 0 4 Submission#4+ColumnDefinition ColumnName Articles DataType int IsNullable True Size 0 5 Submission#4+ColumnDefinition ColumnName Videos DataType int IsNullable True Size 0 6 Submission#4+ColumnDefinition ColumnName Events DataType int IsNullable True Size 0 7 Submission#4+ColumnDefinition ColumnName UsersWithActivity DataType int IsNullable True Size 0 8 Submission#4+ColumnDefinition ColumnName BookGoal DataType int IsNullable True Size 0 9 Submission#4+ColumnDefinition ColumnName ArticleGoal DataType int IsNullable True Size 0 10 Submission#4+ColumnDefinition ColumnName VideoGoal DataType int IsNullable True Size 0 11 Submission#4+ColumnDefinition ColumnName UsersWithGoal DataType int IsNullable True Size 0 3 Submission#4+TableDefinition SchemaName rpt TableName DefaultDateRangeOptions Columns index value 0 Submission#4+ColumnDefinition ColumnName DefaultDateRangeOptionId DataType int IsNullable False Size 0 1 Submission#4+ColumnDefinition ColumnName DateRangeName DataType nvarchar IsNullable True Size 255 2 Submission#4+ColumnDefinition ColumnName MinimumReportPeriod DataType int IsNullable True Size 0 4 Submission#4+TableDefinition SchemaName rpt TableName LearningCampaignEventCo

### Serialize the object to a json file

Adjust the file path to suit.

In [7]:
string path = @".\data\Rpt-Schema-Definitions.json";
internal async Task WriteOutput()
{ 
    await using FileStream output = File.Create(path);
    await System.Text.Json.JsonSerializer.SerializeAsync(output, tableDefinitions);
}
await WriteOutput(); 

## Other approaches

You can also use a predefined library installed locally from a private nuget package.


## Razor Interactive

[Github](https://github.com/plbonneville/RazorInteractive)

This lets us rather nicely generate html from razor pages with embedded C#. The C# and F# Kernel variables are available trough the @Model property


In [8]:
#r "nuget: RazorInteractive"

Installed Packages RazorInteractive, 1.2.0

Loading extensions from `C:\Users\NickMajor\.nuget\packages\razorinteractive\1.2.0\lib\net8.0\RazorInteractive.dll`

Renders the code block as Razor markup in dotnet-interactive notebooks. 
 This extension adds a new kernel that can render Razor markdown. 
 All C# and F# variables are available in the @Model property.

In [9]:
var csColors = new [] { "red", "green", "blue" };

In [10]:
#!razor

<ol>
@foreach(var color in @Model.csColors)
{
    <li style="color: @color;">@color</li>
}
</ol>

red 
 green 
 blue